# Load R libraries

In [ ]:
set.seed(222)
library(monocle3)
library(ggplot2)
library(RColorBrewer)
library(pheatmap)
library(enrichR)
websiteLive <- getOption("enrichR.live")
dbs <- c("GO_Molecular_Function_2023", "GO_Cellular_Component_2023", "GO_Biological_Process_2023",
            "KEGG_2021_Human", "Elsevier_Pathway_Collection", "GWAS_Catalog_2023")
library(dplyr)
library(ggridges)
library(viridis)

# Load functions

## Color settings

In [2]:
source("0_color_settings.r")

## Load trajectory reconstruction and downstream analysis based on Kriegstein scripts

In [3]:
source("0_trajectory_reconstruction_based_on_kriestein.r")

## Scale gene expression into [0, 1]

In [4]:
scale_expression <- function(expr) {
    (expr - min(expr, na.rm = TRUE)) / 
    (max(expr, na.rm = TRUE) - min(expr, na.rm = TRUE))
}

## Obtain comparable fitted values

In [5]:
obtain_fitted_pseudotime_from_compressed_scale_start <- function(cds){
    # Extract the fitted matrices for each lineage
    fitted <- cds@expectation
    
    # Initialize a list to store the adjusted fitted matrices
    adjusted_fitted <- list()
    
    # Loop through each lineage
    for (lineage in names(fitted)) {
        fitted_matrix <- fitted[[lineage]]
        
        # If it's the first lineage, calculate the reference fitted for the first meta-cell
        if (lineage == names(fitted)[1]) {
            reference_fitted <- fitted_matrix[1, ]
        }
        
        # Calculate the difference for each gene
        adjustment <- reference_fitted - fitted_matrix[1, ]
        
        # Adjust the fitted matrix for the lineage
        adjusted_fitted_matrix <- sweep(fitted_matrix, 2, adjustment, '+')
        
        # Add the adjusted matrix to the list
        adjusted_fitted[[lineage]] <- adjusted_fitted_matrix
    }
    cds@expectation <- adjusted_fitted
    return(cds)
}

## Ordered Kmeans clustering based on scaled fitted values

In [6]:
# Function to order rows within a cluster
order_rows_within_cluster <- function(cluster_data) {
    # Calculate row means (excluding the cluster column)
    row_means <- rowMeans(cluster_data[, -ncol(cluster_data)])
    # Order rows based on row means
    ordered_fitted_mat <- cluster_data[order(row_means), ]
    return(ordered_fitted_mat)
}

# color_scale <- colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100)
color_scale <- colorRampPalette(c("#2C7BB6", "white", "#D7191C"))(100)

obtain_peak_times <- function(df, n_metacell){
    n_groups <- ncol(df) / n_metacell
    peak_times <- matrix(NA, nrow(df), n_groups)
    
    for (i in 1:n_groups) {
      cols <- ((i - 1) * 500 + 1):(i * 500)
    
      # Find the index of the max value in each row for these columns
      peak_times[, i] <- apply(df[, cols], 1, which.max)
    }
    
    # Optional: Adjust indices if necessary (e.g., to reflect actual times)
    
    # Calculate the average peak time for each row
    row_averages <- rowMeans(peak_times, na.rm = TRUE)    
}

# Source: https://gist.github.com/mathzero/a2070a24a6b418740c44a5c023f5c01e
save_pheatmap <- function(x, filename, width=12, height=12){
    stopifnot(!missing(x))
    stopifnot(!missing(filename))
    if(grepl(".png",filename)){
        png(filename, width=width, height=height, units = "in", res=300)
        grid::grid.newpage()
        grid::grid.draw(x$gtable)
        dev.off()
    }
    else if(grepl(".pdf",filename)){
        pdf(filename, width=width, height=height)
        grid::grid.newpage()
        grid::grid.draw(x$gtable)
        dev.off()
    }
    else{
      print("Filename did not contain '.png' or '.pdf'")
    }
}


kmeans_cluster_ordered_mean_peak_within <- function(fitted_mat, n_tree, pseudo_range, lineages, branches, n_metacell, n_term, n_char, out_label, plotTextSize=11){
    # Set the maximum value
    # fitted_mat[fitted_mat > quantile(fitted_mat, 0.8)] <- quantile(fitted_mat, 0.8)
    fitted_mat <- t(apply(fitted_mat, 1, scale_expression))
    
    # kmeans clustering
    # 1st
    set.seed(222)
    ks_1st <- kmeans(fitted_mat, centers = n_tree, iter.max = 100)
    
    # prepare for 2nd clustering
    if(length(pseudo_range) == 2){
        peak_times <- abs(apply(fitted_mat, 1, which.max) - n_metacell)
    }else{
        peak_times <- obtain_peak_times(fitted_mat, n_metacell)
    }
    
    # assign each gene to its cluster and add peak times
    gene_info <- data.frame(Cluster = ks_1st$cluster, PeakTime = peak_times)
    # calculate median peak expression time for each cluster
    cluster_peak_times <- aggregate(PeakTime ~ Cluster, data = gene_info, mean)
    # order clusters based on mean peak expression times
    ordered_clusters <- cluster_peak_times[order(cluster_peak_times$PeakTime), "Cluster"]
    
    
    # 2nd: kmeans secondly according to specific order
    set.seed(222)
    centerOrder <- ks_1st$centers[ordered_clusters, ]
    ks_2nd <- kmeans(fitted_mat, centers = centerOrder, iter.max = 100)
    # print(ks_2nd)
    annotation_row <- data.frame(Cluster = factor(ks_2nd$cluster))

    # Ordering the matrix based on the clusters
    # Add cluster information to your fitted_mat
    
    fitted_mat <- cbind(fitted_mat, data.frame(cluster = ks_2nd$cluster))

    # Apply the function to each cluster and combine the results
    ordered_fitted_mat <- do.call(rbind, by(fitted_mat, fitted_mat$cluster, order_rows_within_cluster))
    
    # Annotation for rows
    annotation_row <- data.frame(Cluster = factor(ordered_fitted_mat$cluster))
    rownames(annotation_row) <- rownames(ordered_fitted_mat)
    ordered_fitted_mat$cluster <- NULL

    # Annotation for columns
    annotation_col = data.frame(
                                CellType = factor(rep(lineages, each = n_metacell)), 
                                Branch = factor(branches))
    rownames(annotation_col) = colnames(ordered_fitted_mat)

    
    # Create the heatmap
    par(mfrow = c(1, 1), mar = c(5, 4, 4, 2))
    cols_cluster <- c(brewer.pal(n = n_tree, name = "Paired"))
    names(cols_cluster) <- unique(annotation_row$Cluster)
    ann_colors <- list(CellType = cols_subclass_uni[lineages], Branch = cols_subclass_uni[unique(branches)], Cluster = cols_cluster)
    ph <- pheatmap(ordered_fitted_mat, 
             cluster_rows = F, 
             cluster_cols = F, 
             scale = "none", 
             annotation_row = annotation_row,
             annotation_col = annotation_col,
             annotation_colors = ann_colors,
             show_rownames = F, 
             show_colnames = F, 
             breaks = seq(0, 1, 0.01),
             color = color_scale, 
             border_color = NA)
    save_pheatmap(ph, paste0(out_label, "_ks_", as.character(n_tree), "_heatmap.png"))
    
    # Set the layout for the plots - 1 row and n_tree columns
    pdf(paste0(out_label, "_ks_", as.character(n_tree), "_average_line.pdf"), width = 3 * n_tree, height = 3)
    # options(repr.plot.width = 3 * n_tree, repr.plot.height = 3, repr.plot.res = 300)
    par(mfrow = c(1, n_tree), mar = c(2, 1.8, 2, 1))
    
    for(sub_cluster in seq(1, n_tree, 1)){
        plot(0, type = "n", xlim = c(0, max(pseudo_range)), ylim = c(0, 1), xlab = "Pseudotime", ylab = "Relative expression", main = paste0("Cluster ", sub_cluster, " (", length(ks_2nd$cluster[ks_2nd$cluster == sub_cluster]), ")"))

        # Your lines plotting code
        if(length(pseudo_range) == 2){ 
            lines(seq(0, pseudo_range[1], length.out = n_metacell), 
                  rev(colMeans(fitted_mat[rownames(fitted_mat) %in% names(ks_2nd$cluster[ks_2nd$cluster == sub_cluster]), 1:n_metacell])), 
                  col = cols_subclass_uni[names(pseudo_range[1])], 
                  lwd = 10) 
        } else {
            lines(seq(0, pseudo_range[1], length.out = n_metacell), 
                  colMeans(fitted_mat[rownames(fitted_mat) %in% names(ks_2nd$cluster[ks_2nd$cluster == sub_cluster]), 1:n_metacell]), 
                  col = cols_subclass_uni[names(pseudo_range[1])], 
                  lwd = 10)  
        }
    
        if(length(pseudo_range) >= 2){
            for(i_branch in 2:length(pseudo_range)){
                lines(seq(0, pseudo_range[i_branch], length.out = n_metacell), 
                      colMeans(fitted_mat[rownames(fitted_mat) %in% names(ks_2nd$cluster[ks_2nd$cluster == sub_cluster]), n_metacell*(i_branch-1)+(1:n_metacell)]), 
                      col = cols_subclass_uni[names(pseudo_range)[i_branch]], 
                      lwd = 10)
            } 
        }
    }
    dev.off()

    # Reset default plotting layout
    par(mfrow = c(1, 1), mar = c(5, 4, 4, 2))
    
    # Plot the legend in a new plot
    plot(0, type = "n", axes = FALSE, xlab = "", ylab = "", main = "Legend")
    legend("topleft", legend = names(pseudo_range), col = cols_subclass_uni[names(pseudo_range)], lwd = 6, bty = "n")

    
    
    # Functional analysis
    options(repr.plot.width = 10, repr.plot.height = 7, repr.plot.res = 300)
    enriched_ggplot <- list()
    enriched_terms <- list()
    for(sub_cluster in seq(1, n_tree, 1)){
        GENES <- names(ks_2nd$cluster[ks_2nd$cluster == sub_cluster])
        write.table(unique(GENES), file = paste0(out_label, "_ks_", n_tree, "_cluster_", sub_cluster, ".csv"), sep = ",", row.names = F, col.names = F, quote = F)
        enriched <- enrichr(GENES, dbs)
        for(sub_enrich in names(enriched)){
            all_term <- enriched[[sub_enrich]]
            enriched_ggplot[[sub_enrich]] <- rbind(enriched_ggplot[[sub_enrich]], 
                                       data.frame(Cluster = rep(sub_cluster, n = nrow(all_term)), 
                                              Term = all_term$Term, 
                                              Count = sapply(strsplit(all_term$Genes, ";"), length), 
                                              P.value = all_term$P.value, 
                                              Adjusted.P.value = all_term$Adjusted.P.value,
                                              Neg.Log.Adj.P.Val = -log10(all_term$Adjusted.P.value)))
            top_term <- enriched[[sub_enrich]][order(enriched[[sub_enrich]]$Adjusted.P.value, decreasing = F)[1:n_term], ]
            top_term <- top_term[top_term$Adjusted.P.value < 0.05, ]
            if(nrow(top_term) > 0){
                enriched_terms[[sub_enrich]] <- unique(c(enriched_terms[[sub_enrich]], top_term$Term))               
            }
        }
    }
    
    for(sub_enrich in names(enriched_ggplot)){
        pdf(paste0(out_label, "_ks_", as.character(n_tree), "_func_enrich_", sub_enrich,".pdf"), width = 10, height = 7)
        enriched_data <- enriched_ggplot[[sub_enrich]]
        enriched_data$Cluster <- factor(enriched_data$Cluster, levels = seq(1, n_tree, 1))
        enriched_data <- enriched_data[enriched_data$Term %in% enriched_terms[[sub_enrich]], ]
        enriched_data$Term <- factor(enriched_data$Term, levels = enriched_terms[[sub_enrich]])
        # Add significance label
        enriched_data$plotLabel = ""
        # enriched_data[enriched_data$P.value < 0.05, "plotLabel"] <- "."
        enriched_data[enriched_data$Adjusted.P.value < 0.05, "plotLabel"] <- "#"
        sub_p <- ggplot(enriched_data, aes(x = Cluster, y = Term, size = Count, color = Neg.Log.Adj.P.Val)) +
            geom_point() +
            scale_size(range = c(5, 10)) +
            geom_text(aes(label = plotLabel), size = plotTextSize * 0.55, color = "black") +
            theme_minimal() +
            labs(title = sub_enrich, size = "Count", color = "Neg.Log.Adj.P.Val") +
            scale_color_gradient2(high="#D7191C", mid="white", low="#2C7BB6", name = "-log10(adj.p)")
        print(sub_p)
        dev.off()
    }
    return(list(ks = ks_2nd, fitted_mat = ordered_fitted_mat, enriched = enriched_ggplot))
}

## MAGMA

### Traits used

In [8]:
Psychiatric <- c("sz3", "bip2", "mdd_ipsych", "asd", "adhd_ipsych", "insomn2", "eduAttainment", "intel", "alcoholism_2019", "ocd", "tourette")
Neurological <- c("alzBellenguezNoApoe", "ms", "pd_without_23andMe", "epilepsyFocal", "migraines_2021", "stroke", "als2021")
Others <- c("obesity", "dm2", "lipidCholTotal", "ra", "ibd", "uc")
scDRS_traits <- c(Psychiatric, Neurological, Others)

### Plot LDsc / MAGMA heatmap

In [2]:
# Plot LDsc / MAGMA heatmap
ldscHeatmapPlotter_trait_row <- function(df, plotCol="P", FDR_PER_TRAIT=F, myPadjustMethod="BH", markNominallySignificant=T,plotTextSize=11) {
    df = data.frame(df)
    df$sumstatName <- recode(df$sumstatName, 
                             "sz3"="SCZ",
                             "bip2"="BD",
                             "asd"="ASD",
                             "adhd_ipsych"="ADHD",
                             "mdd_ipsych"="MDD",
                             "ocd"="OCD",
                             "insomn2"="Insomnia",
                             "alcoholism_2019"="Alcoholism",
                             "tourette"="Tourettes",
                             "intel"="IQ",
                             "eduAttainment"="Education",
                             
                             "alzBellenguezNoApoe"="AD",
                             "ms"="MS",
                             "pd_without_23andMe"="PD",
                             "migraines_2021"="Migraines",
                             "als2021"="ALS",
                             "stroke"="Stroke",
                             "epilepsyFocal"="Epilepsy",
                                                        
                             "dm2"="T2D",
                             "ibd"="IBD",
                             "ra"="RArthritis",
                             "lipidCholTotal"="Cholesterol",
                             "obesity"="Obesity",
                             "uc"="UC")
    
    df$plotLabel = ""
    if(markNominallySignificant) {   df$plotLabel[(10^-df[,plotCol]) < 0.05] = "·" }
    df$plotLabel[p.adjust((10^-df[,plotCol]),method=myPadjustMethod) < 0.05]="#"
    
    if(FDR_PER_TRAIT) {
      for(i in unique(df$annoID)) {
        df$adj.P.value[df$annoID==i] = fdrtool(df[,..plotCol][df$annoID==i], statistic=c("pvalue"), cutoff.method=c("fndr"), plot = F)$qval
      }
    }
        
    zz = ggplot(df, aes(annoName, sumstatName, fill = tempScoreCol)) + geom_tile() + scale_y_discrete(expand = c(0, 0)) + scale_x_discrete(expand = c(0, 0)) + xlab("Cluster") + 
      ylab("Trait") + theme_classic(base_size = plotTextSize) + theme(axis.text.x = element_text(colour = "black", angle = -45, hjust = 0)) + 
      coord_fixed() + theme(legend.title = element_text(size = 10, face = "bold"))    
    print(zz + geom_tile(aes(fill = tempScoreCol)) + scale_fill_gradient2(high="#D7191C", mid="white", low="#2C7BB6", name = "-logP") + geom_text(aes(label = plotLabel), size = plotTextSize * 0.55))
}

# DEGs (q_value < 0.05 & Moran >= 0.05)

## Astrocyte

### Load data

In [42]:
cds_AST <- readRDS("aging_lister_integrated_ds1000_rmno_scanpy6000_half_100_123456_beforeUMAT_AST_scanpy6000_25_sparse_123456_afterCombination_cds_compressed.RDS")
cds_AST

class: cell_data_set_ext 
dim: 14613 173241 
metadata(0):
assays(2): counts logcounts
rownames(14613): DPM1 SCYL3 ... OOSP1 C8orf44-SGK3
rowData names(2): gene_name gene_short_name
colnames(173241): H1007-1-AACACACAGTAATTGG-0-Aging
  H1007-1-AACTTCTCATCTCGTC-0-Aging ...
  TTTGTTGAGCGATTCT-RL2132_25yr_v3-Lister
  TTTGTTGCACAAGCTT-RL2132_25yr_v3-Lister
colData names(55): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(1): UMAP
mainExpName: RNA
altExpNames(0):

### DEGs

In [43]:
DEGs_AST <- c()
for(sub_trajectory in names(cds_AST@lineages)){
    data <- read.table(paste0("AST_pt_DEG_", sub_trajectory,".txt"))
    DEGs_AST <- c(DEGs_AST, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_AST <- unique(DEGs_AST)
length(DEGs_AST)

[1] 808

### Obtain fitted matrix

In [ ]:
cds_AST_scale_start <- obtain_fitted_pseudotime_from_compressed_scale_start(cds_AST)
merged_AST_scale_start <- t(rbind(cds_AST_scale_start@expectation[[1]][nrow(cds_AST_scale_start@expectation[[1]]):1, ], cds_AST_scale_start@expectation[[2]]))
merged_AST_scale_start <- merged_AST_scale_start[DEGs_AST, ]
merged_AST_scale_start # 808

### Kmeans clustering (7 groups)

In [ ]:
ks_7_AST_term_3 <- kmeans_cluster_ordered_mean_peak_within(fitted_mat = merged_AST_scale_start, n_tree = 7, 
                                                             lineages = names(cds_AST@lineages), 
                                                             branches = rep(c("Astro_PLSCR1", "Astro_ADAMTSL3"), each = 500), 
                                                             pseudo_range = sapply(cds_AST_scale_start@pseudotime, "max"), 
                                                             n_metacell = 500, n_term = 3, n_char = 100, 
                                                             out_label = "ast_degs")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("MAGMA_AST_ks_7/meta-files/geneSetResults.tsv.gz")

In [24]:
magmasets[magmasets$VARIABLE == "Cluster_1", "VARIABLE"] <- "Early"
magmasets[magmasets$VARIABLE == "Cluster_2", "VARIABLE"] <- "Early-Mid"
magmasets[magmasets$VARIABLE == "Cluster_3", "VARIABLE"] <- "PP_AST-spec All"
magmasets[magmasets$VARIABLE == "Cluster_4", "VARIABLE"] <- "FB_AST-spec Mid"
magmasets[magmasets$VARIABLE == "Cluster_5", "VARIABLE"] <- "Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_6", "VARIABLE"] <- "FB_AST-spec Late"
magmasets[magmasets$VARIABLE == "Cluster_7", "VARIABLE"] <- "Late"


In [ ]:
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% scDRS_traits),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(scDRS_traits))
magmasets$annoName = factor(magmasets$VARIABLE, levels = c("Early", "Early-Mid", "PP_AST-spec All", "FB_AST-spec Mid", "Mid-Late", "FB_AST-spec Late", "Late"))
magmasets$tempScoreCol = magmasets$LogP
                
pdf("ast_degs_ks_7_magma.pdf", width = 7.5, height = 7.5)
ldscHeatmapPlotter_trait_row(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Microglia

### Load data

In [56]:
cds_Micro <- readRDS("aging_lister_integrated_ds1000_rmno_scanpy6000_half_100_123456_beforeUMAT_MICRO_scanpy6000_25_sparse_123456_afterCombination_cds_compressed.RDS")
cds_Micro

class: cell_data_set_ext 
dim: 14479 69463 
metadata(0):
assays(2): counts logcounts
rownames(14479): DPM1 SCYL3 ... TBCE C8orf44-SGK3
rowData names(2): gene_name gene_short_name
colnames(69463): H1007-1-ACCTGTCCATACATCG-0-Aging
  H1007-1-AGACAAAAGACGCAGT-0-Aging ...
  TTTCATGAGAATCGAT-RL2132_25yr_v3-Lister
  TTTGATCTCACTTCTA-RL2132_25yr_v3-Lister
colData names(55): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(1): UMAP
mainExpName: RNA
altExpNames(0):

### DEGs

In [57]:
DEGs_Micro <- c()
for(sub_trajectory in names(cds_Micro@lineages)){
    data <- read.table(paste0("MICRO_pt_DEG_", sub_trajectory,".txt"))
    DEGs_Micro <- c(DEGs_Micro, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_Micro <- unique(DEGs_Micro)
length(DEGs_Micro)

[1] 230

### Obtain fitted matrix

In [ ]:
cds_Micro_scale_start <- obtain_fitted_pseudotime_from_compressed_scale_start(cds_Micro)
merged_Micro_scale_start <- t(do.call(rbind, cds_Micro_scale_start@expectation))
merged_Micro_scale_start <- merged_Micro_scale_start[DEGs_Micro, ]
merged_Micro_scale_start # 230

### Kmeans clustering (3 groups)

In [ ]:
ks_3_Micro_term_3 <- kmeans_cluster_ordered_mean_peak_within(fitted_mat = merged_Micro_scale_start, n_tree = 3, 
                                                             lineages = names(cds_Micro@lineages), 
                                                             branches = rep("Micro", 500),
                                                             pseudo_range = sapply(cds_Micro_scale_start@pseudotime, "max"), 
                                                             n_metacell = 500, n_term = 3, n_char = 100, 
                                                             out_label = "micro_degs")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("MAGMA_MICRO_ks_3/meta-files/geneSetResults.tsv.gz")
magmasets

In [27]:
magmasets[magmasets$VARIABLE == "Cluster_1", "VARIABLE"] <- "Early"
magmasets[magmasets$VARIABLE == "Cluster_2", "VARIABLE"] <- "Mid"
magmasets[magmasets$VARIABLE == "Cluster_3", "VARIABLE"] <- "Late"

In [ ]:
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% scDRS_traits),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(scDRS_traits))
# magmasets$annoName = factor(magmasets$VARIABLE, levels = rev(sort(unique(magmasets$VARIABLE))))
magmasets$annoName = factor(magmasets$VARIABLE, levels = c("Early", "Mid", "Late"))
magmasets$tempScoreCol = magmasets$LogP

pdf("micro_degs_ks_3_magma.pdf", width = 7.5, height = 7.5)
ldscHeatmapPlotter_trait_row(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Oligodendrocyte

### Load data

In [16]:
cds_Oligo <- readRDS("aging_lister_integrated_ds1000_rmno_scanpy6000_half_100_123456_beforeUMAT_OLIGO_scanpy6000_25_sparse_123456_afterCombination_cds_compressed.RDS")
cds_Oligo

class: cell_data_set_ext 
dim: 14666 553620 
metadata(0):
assays(2): counts logcounts
rownames(14666): DPM1 SCYL3 ... OOSP1 C8orf44-SGK3
rowData names(2): gene_name gene_short_name
colnames(553620): H1007-1-AAACGAAGTCTCAGGC-0-Aging
  H1007-1-AAACGAATCCCATAGA-0-Aging ...
  TTTGGAGAGTGGAATT-RL2132_25yr_v3-Lister
  TTTGTTGGTAAGGTCG-RL2132_25yr_v3-Lister
colData names(55): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(1): UMAP
mainExpName: RNA
altExpNames(0):

### DEGs

In [17]:
DEGs_Oligo <- c()
for(sub_trajectory in names(cds_Oligo@lineages)){
    data <- read.table(paste0("OLIGO_pt_DEG_", sub_trajectory,".txt"))
    DEGs_Oligo <- c(DEGs_Oligo, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_Oligo <- unique(DEGs_Oligo)
length(DEGs_Oligo) 

[1] 1531

### Obtain fitted matrix

In [ ]:
cds_Oligo_scale_start <- obtain_fitted_pseudotime_from_compressed_scale_start(cds_Oligo)
merged_Oligo_scale_start <- t(do.call(rbind, cds_Oligo_scale_start@expectation))
merged_Oligo_scale_start <- merged_Oligo_scale_start[DEGs_Oligo, ]
merged_Oligo_scale_start # 1,531

### Kmeans clustering (2 groups)

In [ ]:
ks_2_Oligo_term_3 <- kmeans_cluster_ordered_mean_peak_within(fitted_mat = merged_Oligo_scale_start, n_tree = 2, 
                                                             lineages = names(cds_Oligo@lineages), 
                                                             branches = rep("Oligo", 500),
                                                             pseudo_range = sapply(cds_Oligo_scale_start@pseudotime, "max"), 
                                                             n_metacell = 500, n_term = 3, n_char = 100, 
                                                             out_label = "oligo_degs")


#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("MAGMA_OLIGO_ks_2/meta-files/geneSetResults.tsv.gz")
magmasets

In [10]:
magmasets[magmasets$VARIABLE == "Cluster_1", "VARIABLE"] <- "Early"
magmasets[magmasets$VARIABLE == "Cluster_2", "VARIABLE"] <- "Late"

In [ ]:
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% scDRS_traits),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(scDRS_traits))
# magmasets$annoName = factor(magmasets$VARIABLE, levels = rev(sort(unique(magmasets$VARIABLE))))
magmasets$annoName = factor(magmasets$VARIABLE, levels = c("Early", "Mid", "Late"))
magmasets$tempScoreCol = magmasets$LogP

pdf("oligo_degs_ks_2_magma.pdf", width = 7.5, height = 7.5)
ldscHeatmapPlotter_trait_row(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Excitatory neuron

### Load data

In [64]:
cds_EN <- readRDS("aging_lister_integrated_ds1000_rmno_scanpy6000_half_100_123456_beforeUMAT_EN_scanpy6000_25_sparse_123456_afterCombination_cds_compressed.RDS")
cds_EN

class: cell_data_set_ext 
dim: 14649 349350 
metadata(0):
assays(2): counts logcounts
rownames(14649): DPM1 SCYL3 ... OOSP1 C8orf44-SGK3
rowData names(2): gene_name gene_short_name
colnames(349350): H1007-1-AAACGAAAGTTGTAGA-0-Aging
  H1007-1-AAACGCTAGGAACGTC-0-Aging ...
  TTTGTTGCAACAGTGG-RL2132_25yr_v3-Lister
  TTTGTTGGTTCGGCTG-RL2132_25yr_v3-Lister
colData names(55): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(1): UMAP
mainExpName: RNA
altExpNames(0):

### DEGs

In [65]:
DEGs_EN <- c()
for(sub_trajectory in names(cds_EN@lineages)){
    data <- read.table(paste0("EN_pt_DEG_", sub_trajectory,".txt"))
    DEGs_EN <- c(DEGs_EN, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_EN <- unique(DEGs_EN)
length(DEGs_EN) # 2,388

[1] 2388

### Obtain fitted matrix

In [ ]:
cds_EN_scale_start <- obtain_fitted_pseudotime_from_compressed_scale_start(cds_EN)
merged_EN_scale_start <- t(do.call(rbind, cds_EN_scale_start@expectation))
merged_EN_scale_start <- merged_EN_scale_start[DEGs_EN, ]
merged_EN_scale_start # 2,388

### Kmeans clustering (7 groups)

In [ ]:
ks_7_EN_term_3 <- kmeans_cluster_ordered_mean_peak_within(fitted_mat = merged_EN_scale_start, n_tree = 7, 
                                                             lineages = names(cds_EN@lineages), 
                                                             branches = c(rep("Deep_nonIT", 1500), rep("Deep_IT", 1000), rep("Upper_IT", 2000)), 
                                                             pseudo_range = sapply(cds_EN_scale_start@pseudotime, "max"), 
                                                             n_metacell = 500, n_term = 3, n_char = 100, 
                                                             out_label = "en_degs")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("MAGMA_EN_ks_7/meta-files/geneSetResults.tsv.gz")
magmasets

In [42]:
magmasets[magmasets$VARIABLE == "Cluster_1", "VARIABLE"] <- "Upper-IT-spec Early"
magmasets[magmasets$VARIABLE == "Cluster_2", "VARIABLE"] <- "Deep-layer-spec Early"
magmasets[magmasets$VARIABLE == "Cluster_3", "VARIABLE"] <- "Deep-layer-spec Mid"
magmasets[magmasets$VARIABLE == "Cluster_4", "VARIABLE"] <- "Deep-nonIT-spec Late"
magmasets[magmasets$VARIABLE == "Cluster_5", "VARIABLE"] <- "Deep-nonIT-spec Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_6", "VARIABLE"] <- "Upper-IT-spec Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_7", "VARIABLE"] <- "Mid-Late"

In [ ]:
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% scDRS_traits),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(scDRS_traits))
# magmasets$annoName = factor(magmasets$VARIABLE, levels = rev(sort(unique(magmasets$VARIABLE))))
magmasets$annoName = factor(magmasets$VARIABLE, levels = c("Upper-IT-spec Early", "Deep-layer-spec Early", "Deep-layer-spec Mid", "Deep-nonIT-spec Late", "Deep-nonIT-spec Mid-Late", "Upper-IT-spec Mid-Late", "Mid-Late"))
magmasets$tempScoreCol = magmasets$LogP

pdf("en_degs_ks_7_magma.pdf", width = 7.5, height = 7.5)
ldscHeatmapPlotter_trait_row(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Inhibitory neuron

### Load data

In [68]:
cds_IN <- readRDS("aging_lister_integrated_ds1000_rmno_scanpy6000_half_100_123456_beforeUMAT_IN_scanpy6000_25_sparse_123456_afterCombination_cds_compressed.RDS")
cds_IN

class: cell_data_set_ext 
dim: 14645 254695 
metadata(0):
assays(2): counts logcounts
rownames(14645): DPM1 SCYL3 ... OOSP1 C8orf44-SGK3
rowData names(2): gene_name gene_short_name
colnames(254695): H1007-1-AAAGGATAGAAATGGG-0-Aging
  H1007-1-AAAGGTACATTGCTTT-0-Aging ...
  TTTGTTGCACCGCTGA-RL2132_25yr_v3-Lister
  TTTGTTGTCGTCCTCA-RL2132_25yr_v3-Lister
colData names(55): orig.ident nCount_RNA ... ident Size_Factor
reducedDimNames(1): UMAP
mainExpName: RNA
altExpNames(0):

### DEGs

In [69]:
DEGs_IN <- c()
for(sub_trajectory in names(cds_IN@lineages)){
    data <- read.table(paste0("IN_pt_DEG_", sub_trajectory,".txt"))
    DEGs_IN <- c(DEGs_IN, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_IN <- unique(DEGs_IN)
length(DEGs_IN) # 1,883

[1] 1883

### Obtain fitted matrix

In [ ]:
cds_IN_scale_start <- obtain_fitted_pseudotime_from_compressed_scale_start(cds_IN)
merged_IN_scale_start <- t(do.call(rbind, cds_IN_scale_start@expectation))
merged_IN_scale_start <- merged_IN_scale_start[DEGs_IN, ]
merged_IN_scale_start # 1,883

### Kmeans clustering (7 groups)

In [ ]:
ks_7_IN_term_3 <- kmeans_cluster_ordered_mean_peak_within(fitted_mat = merged_IN_scale_start, n_tree = 7, 
                                                             lineages = names(cds_IN@lineages), 
                                                             branches = c(rep("MGE", 1500), rep("CGE", 3500)), 
                                                             pseudo_range = sapply(cds_IN_scale_start@pseudotime, "max"), 
                                                             n_metacell = 500, n_term = 3, n_char = 100, 
                                                             out_label = "in_degs")


#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("MAGMA_IN_ks_7/meta-files/geneSetResults.tsv.gz")
magmasets

In [4]:
magmasets[magmasets$VARIABLE == "Cluster_1", "VARIABLE"] <- "Early"
magmasets[magmasets$VARIABLE == "Cluster_2", "VARIABLE"] <- "MGE-spec Early"
magmasets[magmasets$VARIABLE == "Cluster_3", "VARIABLE"] <- "IN_PVALB_CHC-spec Early"
magmasets[magmasets$VARIABLE == "Cluster_4", "VARIABLE"] <- "IN_SST-spec Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_5", "VARIABLE"] <- "IN_PVALB_CHC-spec Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_6", "VARIABLE"] <- "CGE-spec Mid-Late"
magmasets[magmasets$VARIABLE == "Cluster_7", "VARIABLE"] <- "Mid-Late"

In [ ]:
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% scDRS_traits),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(scDRS_traits))
# magmasets$annoName = factor(magmasets$VARIABLE, levels = rev(sort(unique(magmasets$VARIABLE))))
magmasets$annoName = factor(magmasets$VARIABLE, levels = c("Early", "MGE-spec Early", "IN_PVALB_CHC-spec Early", "IN_SST-spec Mid-Late", "IN_PVALB_CHC-spec Mid-Late", "CGE-spec Mid-Late", "Mid-Late"))
magmasets$tempScoreCol = magmasets$LogP

pdf("in_degs_ks_7_magma.pdf", width = 7.5, height = 7.5)
ldscHeatmapPlotter_trait_row(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()